# Multi-Agent Resource Allocation via Competitive MultiRound Auctions

In [28]:
import numpy as np
import random
from typing import List, Dict, Tuple, Optional
import pandas as pd

def random_argmax(arr):
    """
    Randomly selects one index among the maximum values in an array.
    """
    max_val = np.max(arr)
    max_indices = np.where(arr == max_val)[0]
    return np.random.choice(max_indices)

class Agent:
    """
    Represents an agent in the auction system.
    """
    def __init__(self, agent_id: int, valuations: List[float]):
        self.agent_id = agent_id
        self.valuations = valuations  # Valuations for each resource
        self.current_bid_resource = None  # Resource currently bidding on
        self.current_bid_amount = 0
        self.eliminated_resources = set()  # Resources eliminated from
        self.no_bid_count = {}  # Track consecutive no-bids per resource
        self.bid_history = []  # Track bidding history

    def reset_round(self):
        """Reset agent state for new round."""
        self.current_bid_resource = None
        self.current_bid_amount = 0

    def can_bid_on_resource(self, resource_id: int) -> bool:
        """Check if agent can bid on a specific resource."""
        return resource_id not in self.eliminated_resources

    def eliminate_from_resource(self, resource_id: int, verbose: bool = False):
        """Eliminate agent from bidding on a specific resource."""
        self.eliminated_resources.add(resource_id)
        if verbose:
            print(f"Agent {self.agent_id} eliminated from Resource {resource_id}")

    def calculate_bid_strategy(self, resource_id: int, current_price: float, epsilon: float) -> float:
        """
        Calculate strategic bid for a resource.
        Strategy: Bid incrementally above current price, but not more than valuation.
        """
        valuation = self.valuations[resource_id]
        min_bid = current_price + epsilon

        # Don't bid more than valuation
        if min_bid > valuation:
            return 0  # Can't afford to bid

        # Strategic bidding: bid slightly above minimum required
        # Add some randomness for different outcomes
        bid_increment = epsilon + random.uniform(0, epsilon * 0.5)
        strategic_bid = min(current_price + bid_increment, valuation)

        return strategic_bid

    def select_resource_to_bid(self, auction_state: Dict, epsilon: float) -> Optional[Tuple[int, float]]:
        """
        Select which resource to bid on and determine bid amount.
        Strategy: Bid on resource with highest potential profit.
        """
        best_resource = None
        best_bid = 0
        best_profit = -1

        for resource_id in range(len(self.valuations)):
            if not self.can_bid_on_resource(resource_id):
                continue

            current_price = auction_state['prices'][resource_id]
            valuation = self.valuations[resource_id]

            # Calculate potential bid
            potential_bid = self.calculate_bid_strategy(resource_id, current_price, epsilon)

            if potential_bid > 0:
                potential_profit = valuation - potential_bid

                # Select resource with highest profit potential
                if potential_profit > best_profit:
                    best_profit = potential_profit
                    best_resource = resource_id
                    best_bid = potential_bid

        if best_resource is not None:
            return (best_resource, best_bid)
        return None

class Auction:
    """
    Manages the multi-round auction process.
    """
    def __init__(self, n_resources: int, m_agents: int, valuations: List[List[float]], epsilon: float):
        self.n_resources = n_resources
        self.m_agents = m_agents
        self.epsilon = epsilon

        # Initialize agents
        self.agents = []
        for j in range(m_agents):
            agent_valuations = [valuations[i][j] for i in range(n_resources)]
            self.agents.append(Agent(j, agent_valuations))

        # Initialize auction state
        self.prices = [0.0] * n_resources  # Current prices for each resource
        self.allocations = [-1] * n_resources  # Which agent owns each resource (-1 = unallocated)
        self.round_number = 0
        self.auction_history = []

    def reset_auction(self):
        """Reset auction to initial state."""
        self.prices = [0.0] * self.n_resources
        self.allocations = [-1] * self.n_resources
        self.round_number = 0
        self.auction_history = []

        # Reset all agents
        for agent in self.agents:
            agent.eliminated_resources = set()
            agent.no_bid_count = {}
            agent.bid_history = []

    def get_auction_state(self) -> Dict:
        """Get current auction state."""
        return {
            'prices': self.prices.copy(),
            'allocations': self.allocations.copy(),
            'round': self.round_number
        }

    def process_round_bids(self, verbose: bool = False) -> bool:
        """
        Process all bids for the current round.
        Returns True if any changes occurred, False if auction should end.
        """
        # Reset agents for new round
        for agent in self.agents:
            agent.reset_round()

        # Collect bids from all agents
        round_bids = {}  # resource_id -> [(agent_id, bid_amount), ...]

        if verbose:
            print(f"\n--- Round {self.round_number + 1} Bid Collection ---")
            print(f"Current prices: {[f'{p:.2f}' for p in self.prices]}")
            print(f"Current allocations: {self.allocations}")

        for agent in self.agents:
            bid_info = agent.select_resource_to_bid(self.get_auction_state(), self.epsilon)

            if bid_info:
                resource_id, bid_amount = bid_info
                agent.current_bid_resource = resource_id
                agent.current_bid_amount = bid_amount

                if resource_id not in round_bids:
                    round_bids[resource_id] = []
                round_bids[resource_id].append((agent.agent_id, bid_amount))

                # Record bid in history
                agent.bid_history.append({
                    'round': self.round_number,
                    'resource': resource_id,
                    'bid': bid_amount
                })

                if verbose:
                    print(f"Agent {agent.agent_id} bids {bid_amount:.2f} on Resource {resource_id} (valuation: {agent.valuations[resource_id]:.2f})")
            else:
                if verbose:
                    print(f"Agent {agent.agent_id} makes no bid this round")

        if verbose and not round_bids:
            print("No bids submitted this round")

        # Process bids for each resource
        changes_made = False

        if verbose and round_bids:
            print(f"\n--- Round {self.round_number + 1} Bid Processing ---")

        for resource_id in range(self.n_resources):
            if resource_id in round_bids:
                bids = round_bids[resource_id]

                if verbose:
                    print(f"\nResource {resource_id} bids: {[(f'Agent {bid[0]}', f'{bid[1]:.2f}') for bid in bids]}")

                # Find highest bid(s)
                max_bid = max(bid[1] for bid in bids)
                max_bidders = [bid for bid in bids if bid[1] == max_bid]

                # Handle ties randomly
                if len(max_bidders) > 1:
                    winner_idx = random.randint(0, len(max_bidders) - 1)
                    winning_agent, winning_bid = max_bidders[winner_idx]
                    if verbose:
                        print(f"Tie between {len(max_bidders)} agents, randomly selected Agent {winning_agent}")
                else:
                    winning_agent, winning_bid = max_bidders[0]

                # Check if bid is valid (higher than current price + epsilon)
                if winning_bid >= self.prices[resource_id] + self.epsilon:
                    # Update allocation and price
                    old_allocation = self.allocations[resource_id]
                    self.allocations[resource_id] = winning_agent
                    self.prices[resource_id] = winning_bid
                    changes_made = True

                    if verbose:
                        if old_allocation != -1:
                            print(f"Resource {resource_id} reallocated: Agent {old_allocation} → Agent {winning_agent} at price {winning_bid:.2f}")
                        else:
                            print(f"Resource {resource_id} allocated: Agent {winning_agent} at price {winning_bid:.2f}")
                else:
                    if verbose:
                        print(f"Bid {winning_bid:.2f} invalid for Resource {resource_id} (minimum required: {self.prices[resource_id] + self.epsilon:.2f})")
            else:
                if verbose:
                    print(f"No bids for Resource {resource_id}")

        # Update no-bid counts and handle eliminations
        self.update_no_bid_counts(round_bids, verbose)

        # Record round in history
        self.auction_history.append({
            'round': self.round_number,
            'bids': round_bids,
            'allocations': self.allocations.copy(),
            'prices': self.prices.copy()
        })

        if verbose:
            print(f"\nRound {self.round_number + 1} summary:")
            print(f"  Changes made: {changes_made}")
            print(f"  Final allocations: {self.allocations}")
            print(f"  Final prices: {[f'{p:.2f}' for p in self.prices]}")

        return changes_made

    def update_no_bid_counts(self, round_bids: Dict, verbose: bool = False):
        """Update no-bid counts and eliminate agents if necessary."""
        eliminations_this_round = []

        for agent in self.agents:
            for resource_id in range(self.n_resources):
                if resource_id not in agent.eliminated_resources:
                    # Check if agent bid on this resource
                    agent_bid_on_resource = (
                        resource_id in round_bids and
                        any(bid[0] == agent.agent_id for bid in round_bids[resource_id])
                    )

                    if not agent_bid_on_resource:
                        # Increment no-bid count
                        if resource_id not in agent.no_bid_count:
                            agent.no_bid_count[resource_id] = 0
                        agent.no_bid_count[resource_id] += 1

                        # Eliminate if no bid for 2 consecutive rounds
                        if agent.no_bid_count[resource_id] >= 2:
                            agent.eliminate_from_resource(resource_id, verbose)
                            eliminations_this_round.append((agent.agent_id, resource_id))
                    else:
                        # Reset no-bid count if agent bid
                        agent.no_bid_count[resource_id] = 0

        if verbose and eliminations_this_round:
            print(f"\nEliminations this round: {eliminations_this_round}")

    def run_auction(self, verbose: bool = False) -> Tuple[List[int], List[float]]:
        """
        Run the complete auction process.
        Returns final allocations and prices.
        """
        if verbose:
            print(f"\n=== Starting Auction ===")
            print(f"Resources: {self.n_resources}, Agents: {self.m_agents}, Epsilon: {self.epsilon}")
            print("Agent valuations:")
            for agent in self.agents:
                print(f"  Agent {agent.agent_id}: {[f'{v:.2f}' for v in agent.valuations]}")

        max_rounds = 50  # Prevent infinite loops

        while self.round_number < max_rounds:
            if verbose:
                print(f"\n--- Round {self.round_number + 1} ---")

            changes_made = self.process_round_bids(verbose)
            self.round_number += 1

            if not changes_made:
                if verbose:
                    print(f"\n=== Auction Ended ===")
                    print(f"No changes in round {self.round_number} - auction complete")
                break

        if verbose and self.round_number >= max_rounds:
            print(f"\n=== Auction Ended ===")
            print(f"Maximum rounds ({max_rounds}) reached")

        return self.allocations, self.prices

    def print_final_results(self):
        """Print final allocation results."""
        for resource_id in range(self.n_resources):
            if self.allocations[resource_id] != -1:
                agent_id = self.allocations[resource_id]
                price = self.prices[resource_id]
                print(f"Resource {resource_id + 1} → Agent {agent_id + 1} at price {price:.2f}")
            else:
                print(f"Resource {resource_id + 1} → Unallocated")

def run_multiple_auctions(n_resources: int, m_agents: int, valuations: List[List[float]],
                         epsilon: float, num_runs: int = 5, verbose: bool = False):
    """
    Run multiple auction instances and compare results.
    """

    results = []

    for run in range(num_runs):
        print(f"{'='*20} RUN {run + 1} {'='*20}")

        # Create new auction instance
        auction = Auction(n_resources, m_agents, valuations, epsilon)

        # Run auction
        allocations, prices = auction.run_auction(verbose=verbose)

        # Print results
        auction.print_final_results()

        # Store results
        results.append({
            'run': run + 1,
            'allocations': allocations.copy(),
            'prices': prices.copy()
        })

    return results


# Example Input

In [25]:
print("Example Input:")
# Example input
n = 2  # Number of resources
m = 3  # Number of agents
V = [
    [50, 80, 70],  # Valuations of agents for resource 1
    [60, 90, 30]   # Valuations of agents for resource 2
]
ε = 10  # Minimum bid increment

# Run multiple auctions with verbose output
results = run_multiple_auctions(n, m, V, ε, num_runs=5, verbose=False)


Example Input:
==================== RUN 1 ====================
Resource 1 → Agent 3 at price 65.71
Resource 2 → Agent 2 at price 80.08
==================== RUN 2 ====================
Resource 1 → Agent 3 at price 63.44
Resource 2 → Agent 2 at price 90.00
==================== RUN 3 ====================
Resource 1 → Agent 3 at price 60.38
Resource 2 → Agent 2 at price 87.09
==================== RUN 4 ====================
Resource 1 → Agent 3 at price 69.46
Resource 2 → Agent 2 at price 81.52
==================== RUN 5 ====================
Resource 1 → Agent 3 at price 67.24
Resource 2 → Agent 2 at price 82.15


# Random Parameter Generation:


1.   Random number of resources: Between 2-5 resources
2.   Random number of agents: Between 2-6 agents
3.   Random valuations: Each agent's valuation for each resource is randomly
4.   generated between 10-100
5.   Random epsilon: Minimum bid increment between 5-15






In [27]:
# Generate random parameters
n = random.randint(2, 5)  # Random number of resources (2-5)
m = random.randint(2, 6)  # Random number of agents (2-6)

# Generate random valuations matrix
# Each agent's valuation for each resource is between 10 and 100
V = []
for resource_id in range(n):
    resource_valuations = []
    for agent_id in range(m):
        valuation = random.randint(10, 100)
        resource_valuations.append(valuation)
    V.append(resource_valuations)

# Random epsilon between 5 and 15
ε = random.randint(5, 15)

print(f"Number of resources: {n}")
print(f"Number of agents: {m}")
print(f"Minimum bid increment (ε): {ε}")
print("\nAgent valuations for each resource:")
for i, resource_vals in enumerate(V):
    print(f"Resource {i + 1}: {resource_vals}")

print(f"\nAgent-wise valuations:")
for agent_id in range(m):
    agent_vals = [V[resource_id][agent_id] for resource_id in range(n)]
    print(f"Agent {agent_id + 1}: {agent_vals}")

# Run multiple auctions with verbose output
results = run_multiple_auctions(n, m, V, ε, num_runs=5, verbose=False)

Number of resources: 4
Number of agents: 6
Minimum bid increment (ε): 15

Agent valuations for each resource:
Resource 1: [96, 58, 60, 35, 19, 85]
Resource 2: [98, 90, 41, 23, 99, 48]
Resource 3: [97, 86, 25, 82, 15, 54]
Resource 4: [78, 64, 94, 57, 18, 74]

Agent-wise valuations:
Agent 1: [96, 98, 97, 78]
Agent 2: [58, 90, 86, 64]
Agent 3: [60, 41, 25, 94]
Agent 4: [35, 23, 82, 57]
Agent 5: [19, 99, 15, 18]
Agent 6: [85, 48, 54, 74]
==================== RUN 1 ====================
Resource 1 → Agent 1 at price 91.02
Resource 2 → Agent 5 at price 94.18
Resource 3 → Agent 4 at price 78.41
Resource 4 → Agent 3 at price 87.26
==================== RUN 2 ====================
Resource 1 → Agent 6 at price 72.27
Resource 2 → Agent 5 at price 94.87
Resource 3 → Agent 1 at price 97.00
Resource 4 → Agent 3 at price 91.23
==================== RUN 3 ====================
Resource 1 → Agent 1 at price 93.79
Resource 2 → Agent 5 at price 99.00
Resource 3 → Agent 4 at price 76.10
Resource 4 → Agent 3 a